In [1]:
import pandas as pd, numpy as np

# LSFF: basic data cleaning

IN: extraction sheet + vehicle-nutrient, country-vehicle dicts

OUT: cleaned extraction sheet. values renamed, dropped all illegal location-vehicle-val-nutrient combos

In [2]:
## load targets
import pickle
inputs_dir = '/ihme/homes/beatrixh/vivarium_data_analysis/pre_processing/lsff_project/data_prep/inputs/'

with open(inputs_dir + 'lsff_vehicle_nutrient_pairs.pickle', 'rb') as handle:
    vehicle_nutrient_map = pickle.load(handle)
    
with open(inputs_dir + 'lsff_country_vehicle_pairs.pickle', 'rb') as handle:
    country_vehicle_map = pickle.load(handle)

countries = [i for i in country_vehicle_map.keys() if type(i)==str]

path = inputs_dir + 'extraction_sheet_lsff_02_16_2021.csv'
df = pd.read_csv(path, encoding = 'utf-8')

In [3]:
path = inputs_dir + 'extraction_sheet_lsff_02_16_2021.csv'
df = pd.read_csv(path, encoding = 'utf-8')

In [4]:
check_cols = ['location_id','location_name','vehicle','value_description','nutrient','value_mean','value_025_percentile',
       'value_975_percentile','notes']

In [5]:
# format vars
df.location_id = df.location_id.fillna(-1).astype(int)
df.location_name = df.location_name.astype(str)
df.vehicle = df.vehicle.astype(str)
df.value_description = df.value_description.astype(str)

df.value_mean = df.value_mean.mask(df.value_mean.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)
df.value_025_percentile = df.value_025_percentile.mask(df.value_025_percentile.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)
df.value_975_percentile = df.value_975_percentile.mask(df.value_975_percentile.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)

In [6]:
#drop all NaN-only rows
df = df.loc[~((df.value_mean.isna())&(df.value_025_percentile.isna())&(df.value_975_percentile.isna()))]

In [7]:
#locations for which need tofill in loc_ids
df[df.location_id==-1].location_name.unique()

array(['China', 'Egypt', 'South Africa', 'Nepal'], dtype=object)

In [8]:
#looks like we'll need to estimate mean here
df.loc[df.value_mean.isna(),check_cols]

,location_id,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,notes
262,207,Ghana,rice,percent of vehicle that is industrially produced,na,NaN,11.0,23.0,"% industrially milled, only a range was given"
501,214,Nigeria,rice,percent of vehicle that is industrially produced,na,NaN,12.0,24.0,"% industrially milled, only a range was given"


## location_name

In [9]:
df.location_name.unique()

array(['India', 'Bangladesh', 'Pakistan', 'Burkina Faso', 'Nigeria',
       "Côte d'Ivoire", 'Cameroon', 'Uganda',
       'United Republic of Tanzania', 'Ethiopia', 'China',
       'Democratic Republic of the Congo', 'Egypt', 'Philippines',
       'Ghana', 'Niger', 'Sudan', 'Yemen', 'Viet Nam', 'Angola', 'Kenya',
       'Indonesia', 'Madagascar', 'Mozambique', 'Nepal', 'Myanmar',
       'South Africa', "CÃ´te d'Ivoire"], dtype=object)

In [10]:
relabel_location_name = {'India': 'India',
 'Bangladesh': 'Bangladesh',
 'Cameroon': 'Cameroon',
 'Nigeria': 'Nigeria',
 'Pakistan': 'Pakistan',
 'Burkina Faso': 'Burkina Faso',
 "CÃ´te d'Ivoire": "Côte d'Ivoire",
 "Côte d'Ivoire":"Côte d'Ivoire",
 'Uganda': 'Uganda',
 'United Republic of Tanzania': 'United Republic of Tanzania',
 'Ethiopia': 'Ethiopia',
 'China': 'China',
 'Indonesia': 'Indonesia',
 'Democratic Republic of the Congo': 'Democratic Republic of the Congo',
 'Kenya': 'Kenya',
 'Egypt': 'Egypt',
 'Niger': 'Niger',
 'Philippines': 'Philippines',
 'Ghana': 'Ghana',
 'Afghanistan': 'Afghanistan',
 'Viet Nam': 'Vietnam', ##TODO: fix this so that it's Viet Nam
 'Yemen': 'Yemen',
 'Sudan': 'Sudan',
 'Angola': 'Angola',
 'Iraq':'Iraq',
 'Madagascar': 'Madagascar',
 'Mozambique': 'Mozambique',
 'Nepal': 'Nepal',
 'Myanmar': 'Myanmar',
 'South Africa': 'South Africa'}

In [11]:
df.location_name = df.location_name.map(relabel_location_name)

In [12]:
## exclude countries the gates foundation had us drop (politically unstable or low rates of maternal and child anemia)
excluded_countries = ['Afghanistan','Yemen','Philippines','Iraq']
df = df[~df.location_name.isin(excluded_countries)]

In [13]:
## clean country names
assert df.location_name.nunique()==25, "wrong number of countries"

In [14]:
df.location_name.unique()

array(['India', 'Bangladesh', 'Pakistan', 'Burkina Faso', 'Nigeria',
       "Côte d'Ivoire", 'Cameroon', 'Uganda',
       'United Republic of Tanzania', 'Ethiopia', 'China',
       'Democratic Republic of the Congo', 'Egypt', 'Ghana', 'Niger',
       'Sudan', 'Vietnam', 'Angola', 'Kenya', 'Indonesia', 'Madagascar',
       'Mozambique', 'Nepal', 'Myanmar', 'South Africa'], dtype=object)

In [15]:
for i in df.location_name.unique():
    if i not in countries:
        print(i)
    assert(i in countries)

## vehicle

In [16]:
##this one looks fine
df.vehicle.unique()

array(['oil', 'wheat flour', 'salt', 'rice', 'maize flour', 'bouillon'],
      dtype=object)

## nutrient

In [17]:
nutrient_map = {'vitamin a': 'vitamin a',
 'na': 'na',
 'iron': 'iron',
 'folic acid, folate, b9': 'folic acid',
 'b12': 'vitamin b12',
 'iodine': 'iodine',
 'b1, thiamine': 'vitamin b1',
 'zinc': 'zinc',
 'd, ergocalciferol-D2, cholecalciferol-D3, alfacalcidol': 'vitamin d',
 'b2, riboflavin': 'vitamin b2',
 'b3, niacin': 'vitamin b3',
 'b6, pyridoxine': 'vitamin b6'}

In [18]:
df.nutrient = df.nutrient.map(nutrient_map)

In [19]:
# fill nans
df['nutrient'] = df['nutrient'].fillna('na')

## value_description

In [20]:
df.value_description.unique()

array(['percent of population eating fortified vehicle',
       'percent of population eating industrially produced vehicle',
       'percent of population eating vehicle',
       'percent of vehicle that is industrially produced',
       'percent of vehicle that is fortified',
       'percent of marketshare of fortified products'], dtype=object)

## eliminate country-vehicle and vehicle-nutrient pairs we're uninterested in

In [21]:
country_vehicle_pairs = pd.DataFrame([(i,j) for i in countries for j in country_vehicle_map[i]], columns = ['location_name','vehicle'])

In [22]:
vehicles = list(vehicle_nutrient_map.keys())
vehicle_nutrient_pairs = pd.DataFrame([(i,j) for i in vehicles for j in vehicle_nutrient_map[i]], columns = ['vehicle','nutrient'])
vehicle_nutrient_pairs = pd.concat([vehicle_nutrient_pairs, pd.DataFrame([(i,'na') for i in vehicles], columns = ['vehicle','nutrient'])])

In [23]:
# check what we're removing
test = df.merge(vehicle_nutrient_pairs, on = ['vehicle','nutrient'], how = 'right')
[(i,j) for (i,j) in zip(df.vehicle,df.nutrient) if (i,j) not in zip(test.vehicle,test.nutrient)]

[('salt', 'iodine'),
 ('salt', 'iodine'),
 ('salt', 'iodine'),
 ('salt', 'iodine'),
 ('salt', 'iodine'),
 ('salt', 'iodine'),
 ('rice', 'vitamin a'),
 ('salt', 'iodine')]

In [24]:
df = df.merge(country_vehicle_pairs, on = ['location_name','vehicle'], how = 'right') #should this be inner?

In [25]:
df = df.merge(vehicle_nutrient_pairs, on = ['vehicle','nutrient'], how = 'right')

In [26]:
#drop all NaN-only rows
df = df.loc[~((df.value_mean.isna())&(df.value_025_percentile.isna())&(df.value_975_percentile.isna()))]

## clean illegal value-nutrient combos

In [27]:
nutrient_irrelevant_vds = ['percent of population eating industrially produced vehicle',
       'percent of population eating vehicle',
       'percent of vehicle that is industrially produced',
       'percent of marketshare of fortified products']

nutrient_relevant_vds = ['percent of population eating fortified vehicle',
       'percent of vehicle that is fortified',
       'percent of marketshare of fortified products']

In [28]:
df.nutrient.unique()

array(['vitamin a', 'na', 'vitamin d', 'iron', 'folic acid',
       'vitamin b12', 'zinc', 'vitamin b1'], dtype=object)

In [29]:
df.loc[(df.value_description.isin(nutrient_irrelevant_vds))&(df.nutrient!='na'),check_cols]

,location_id,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,notes
1,161.0,Bangladesh,oil,percent of vehicle that is industrially produced,vitamin a,52.0,NaN,NaN,no uncertainty
3,161.0,Bangladesh,oil,percent of population eating industrially prod...,vitamin a,87.5,NaN,NaN,no uncertainty
17,205.0,Côte d'Ivoire,oil,percent of marketshare of fortified products,vitamin a,89.0,NaN,NaN,NaN


In [30]:
# relabel nutrients for rows for which nutrient doesn't apply
df.loc[(df.value_description.isin(nutrient_irrelevant_vds))&(df.nutrient!='na'),'nutrient'] = 'na'

In [31]:
# these need to be re extracted; for now we'll drop them
df.loc[(df.value_description.isin(nutrient_relevant_vds)) & (df.nutrient=='na'),check_cols + ['user']].sort_values(['user','location_name'])

,location_id,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,notes,user
509,202.0,Cameroon,maize flour,percent of vehicle that is fortified,na,50.0,NaN,NaN,Unknown what this measure of fortified maize f...,paulina
17,205.0,Côte d'Ivoire,oil,percent of marketshare of fortified products,na,89.0,NaN,NaN,NaN,paulina


In [32]:
# drop rows that need nutrient filled in
df = df.loc[~((df.value_description.isin(nutrient_relevant_vds)) & (df.nutrient=='na')),]

In [33]:
# format vars
df.location_id = df.location_id.fillna(-1).astype(int)
df.location_name = df.location_name.astype(str)
df.vehicle = df.vehicle.astype(str)
df.value_description = df.value_description.astype(str)

df.value_mean = df.value_mean.mask(df.value_mean.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)
df.value_025_percentile = df.value_025_percentile.mask(df.value_025_percentile.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)
df.value_975_percentile = df.value_975_percentile.mask(df.value_975_percentile.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)

In [34]:
df.head()

,location_id,location_name,subnational_name,subnational_location_id,urbanicity,vehicle,nutrient,nutrient_compound,nutrient_mass_ppm,fortification_standards,...,source_citation,source_link,source_year,source_type,source_additional,notes,user,date_recorded,Validated,Unnamed: 27
0,163,India,Rajasthan,NaN,NaN,oil,vitamin a,unknown,7.5,voluntary,...,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,2017,NaN,NaN,NaN,nathaniel,10/30/2020,Yes,NaN
1,161,Bangladesh,na,NaN,NaN,oil,na,na,22.5,mandatory,...,GFDx,https://fortificationdata.org/country-fortific...,2018,NaN,NaN,no uncertainty,yaqi,11/18/2020,NaN,NaN
2,161,Bangladesh,na,NaN,NaN,oil,vitamin a,na,22.5,mandatory,...,GFDx,https://fortificationdata.org/country-fortific...,2017,NaN,NaN,no uncertainty,yaqi,11/18/2020,NaN,NaN
3,161,Bangladesh,na,NaN,NaN,oil,na,na,na,mandatory,...,"Raghavan R, Aaron GJ, Nahar B, et al. Househol...",https://journals.plos.org/plosone/article?id=1...,2015,NaN,NaN,no uncertainty,yaqi,11/18/2020,NaN,NaN
4,165,Pakistan,Balochistan,NaN,unknown,oil,vitamin a,na,NaN,mandatory,...,NaN,https://www.gainhealth.org/sites/default/files...,NaN,Survey - cross-sectional,NaN,Consumption of the fortified food vehicle was ...,rohit,11/24/2020,NaN,NaN


In [35]:
df[df.location_id==-1]

,location_id,location_name,subnational_name,subnational_location_id,urbanicity,vehicle,nutrient,nutrient_compound,nutrient_mass_ppm,fortification_standards,...,source_citation,source_link,source_year,source_type,source_additional,notes,user,date_recorded,Validated,Unnamed: 27
145,-1,Nepal,na,NaN,mixed/both,oil,na,na,na,na,...,GHDx,http://internal-ghdx.healthdata.org/record/nep...,2016,Survey - cross-sectional,NaN,Reported Households Used Cooking Oil to Cook o...,paulina,12/09/2020,NaN,NaN
146,-1,Nepal,na,NaN,urban,oil,na,na,na,na,...,GHDx,http://internal-ghdx.healthdata.org/record/nep...,2016,Survey - cross-sectional,NaN,Reported Households Used Cooking Oil to Cook o...,paulina,12/09/2020,NaN,NaN
147,-1,Nepal,na,NaN,rural,oil,na,na,na,na,...,GHDx,http://internal-ghdx.healthdata.org/record/nep...,2016,Survey - cross-sectional,NaN,Reported Households Used Cooking Oil to Cook o...,paulina,12/09/2020,NaN,NaN
166,-1,China,na,NaN,NaN,wheat flour,iron,NaN,NaN,voluntary,...,GFDx,https://fortificationdata.org/country-fortific...,NaN,NaN,NaN,NaN,paulina,12/04/2020,NaN,NaN
248,-1,China,na,NaN,NaN,wheat flour,na,na,NaN,voluntary,...,NaN,https://www.mdpi.com/2072-6643/11/7/1594/htm,2012,Survey - cross-sectional,NaN,Data is in table S1,rohit,12/18/2020,NaN,NaN
249,-1,China,na,NaN,unknown,wheat flour,na,na,NaN,voluntary,...,NaN,https://fortificationdata.org/chart-quantity-a...,NaN,Survey - cross-sectional,NaN,NaN,rohit,12/10/2020,NaN,NaN
252,-1,Egypt,na,NaN,NaN,wheat flour,na,na,NaN,mandatory,...,NaN,https://www.mdpi.com/2072-6643/11/7/1594/htm,NaN,Survey - cross-sectional,NaN,Data is in table S2 and they did not mention year,rohit,12/18/2020,NaN,NaN
253,-1,Egypt,na,NaN,NaN,wheat flour,na,na,na,na,...,GFDx,https://fortificationdata.org/country-fortific...,2017,NaN,NaN,Source for industrially processed: Magdy Sheha...,paulina,12/04/2020,NaN,NaN
295,-1,Nepal,na,NaN,mixed/both,wheat flour,na,na,na,na,...,GHDx,http://internal-ghdx.healthdata.org/record/nep...,2016,Survey - cross-sectional,NaN,Specifically Maida Wheat Flour Purchased and A...,paulina,12/09/2020,NaN,NaN
296,-1,Nepal,na,NaN,urban,wheat flour,na,na,na,na,...,GHDx,http://internal-ghdx.healthdata.org/record/nep...,2016,Survey - cross-sectional,NaN,Specifically Maida Wheat Flour Purchased and A...,paulina,12/09/2020,NaN,NaN


In [36]:
loc_ids_path = '/ihme/homes/beatrixh/notebooks/viv_rsc/data_prep/inputs/location_ids.csv'
loc_ids = pd.read_csv(loc_ids_path)
loc_ids.location_name = loc_ids.location_name.astype(str)

In [37]:
loc_ids[loc_ids.location_name=="Viet Nam"]

,location_id,location_name
542,20,Viet Nam


In [38]:
ref = [str(i) for i in loc_ids.location_name]

In [39]:
df.loc[df.location_name=='Vietnam','location_name'] = 'Viet Nam' # FIX THIS AT THE ORIGIN

In [40]:
for loc in df.location_name.unique():
    if loc not in ref:
        print(loc)

In [41]:
df.head()

,location_id,location_name,subnational_name,subnational_location_id,urbanicity,vehicle,nutrient,nutrient_compound,nutrient_mass_ppm,fortification_standards,...,source_citation,source_link,source_year,source_type,source_additional,notes,user,date_recorded,Validated,Unnamed: 27
0,163,India,Rajasthan,NaN,NaN,oil,vitamin a,unknown,7.5,voluntary,...,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,2017,NaN,NaN,NaN,nathaniel,10/30/2020,Yes,NaN
1,161,Bangladesh,na,NaN,NaN,oil,na,na,22.5,mandatory,...,GFDx,https://fortificationdata.org/country-fortific...,2018,NaN,NaN,no uncertainty,yaqi,11/18/2020,NaN,NaN
2,161,Bangladesh,na,NaN,NaN,oil,vitamin a,na,22.5,mandatory,...,GFDx,https://fortificationdata.org/country-fortific...,2017,NaN,NaN,no uncertainty,yaqi,11/18/2020,NaN,NaN
3,161,Bangladesh,na,NaN,NaN,oil,na,na,na,mandatory,...,"Raghavan R, Aaron GJ, Nahar B, et al. Househol...",https://journals.plos.org/plosone/article?id=1...,2015,NaN,NaN,no uncertainty,yaqi,11/18/2020,NaN,NaN
4,165,Pakistan,Balochistan,NaN,unknown,oil,vitamin a,na,NaN,mandatory,...,NaN,https://www.gainhealth.org/sites/default/files...,NaN,Survey - cross-sectional,NaN,Consumption of the fortified food vehicle was ...,rohit,11/24/2020,NaN,NaN


In [42]:
df = df.drop(columns=['location_id'])

In [43]:
df = loc_ids.merge(df, on = 'location_name', how = 'right')

In [44]:
df.head()

,location_id,location_name,subnational_name,subnational_location_id,urbanicity,vehicle,nutrient,nutrient_compound,nutrient_mass_ppm,fortification_standards,...,source_citation,source_link,source_year,source_type,source_additional,notes,user,date_recorded,Validated,Unnamed: 27
0,141,Egypt,na,NaN,NaN,wheat flour,na,na,NaN,mandatory,...,NaN,https://www.mdpi.com/2072-6643/11/7/1594/htm,NaN,Survey - cross-sectional,NaN,Data is in table S2 and they did not mention year,rohit,12/18/2020,NaN,NaN
1,141,Egypt,na,NaN,NaN,wheat flour,na,na,na,na,...,GFDx,https://fortificationdata.org/country-fortific...,2017,NaN,NaN,Source for industrially processed: Magdy Sheha...,paulina,12/04/2020,NaN,NaN
2,141,Egypt,na,NaN,NaN,maize flour,na,na,na,na,...,GFDx,https://fortificationdata.org/country-fortific...,2017,NaN,NaN,Source for industrially processed: Magdy Sheha...,paulina,12/04/2020,NaN,NaN
3,522,Sudan,na,NaN,mixed/both,oil,vitamin a,na,NaN,unknown,...,GFDx,https://fortificationdata.org/country-fortific...,2019,NaN,NaN,NaN,beatrix,11/30/2020,NaN,NaN
4,522,Sudan,na,NaN,mixed/both,oil,na,na,NaN,unknown,...,GFDx,https://fortificationdata.org/country-fortific...,2019,NaN,NaN,NaN,beatrix,11/30/2020,NaN,NaN


In [45]:
assert(len(df[df.location_id.isna()])==0), "location_ids didn't merge properly"

## save 

In [47]:
df.loc[df.location_name=='Viet Nam','location_name'] = 'Vietnam' # FIX THIS AT THE ORIGIN

In [48]:
save_path = '/ihme/homes/beatrixh/notebooks/viv_rsc/data_prep/outputs/lsff_data_stage0.csv'
df.to_csv(save_path, index = False)